# <b>Simulation in Python</b>

### Simulation

In [2]:
#Code700
import numpy as np
import random
import math


def expo(lmbd):
    u=random.random()
    return ((-1/lmbd)*math.log(u))

#rename this function and add comments
def dox(pevent, psim): #  pnx, pevt, pnb, pn, pclock, ptx, ptd, ptau, pmaxT=0):

    pmaxT = psim["dpr"]["max"]
    psim[pevent]["nx"] += 1
    psim[pevent]["ex"].append(psim[pevent]["tx"])
    if(pmaxT == 0 or psim["n"] >= 0):
        psim["nb"][psim["n"]] = (psim["nb"][psim["n"]] \
                                 if(psim["n"] in psim["nb"]) else 0) + \
                                    psim[pevent]["tx"] - psim["clock"]
    psim["n"] += (1 if pevent == "arv" else -1)
    psim["clock"] = psim[pevent]["tx"]
    psim[pevent]["tx"] = (psim["clock"] + expo(psim[pevent]["taux"])) \
        if(pmaxT == 0 or psim["n"]>0) else pmaxT

    if(psim["n"] == 1): psim["dpr"]["tx"] = psim["clock"] + expo(psim["dpr"]["taux"])

    print(str(psim["clock"]) + " \t " + str(psim["n"]) + " \t " + str(psim[pevent]["nx"]))

#
def simul(tau1,tau2,maxT):
    #
    sim = {
        "n"   : 0, "clock": 0, "nb" : {},
        "arv" : { "tx" : expo(tau1), "ex": [], "nx" : 0, "taux": tau1 },
        "dpr" : { "tx" : maxT      , "ex": [], "nx" : 0, "taux": tau2  , "max":maxT}, 
        #next arrival time, next departure time
        }

    #
    while(sim["clock"] < maxT):
        dox("arv" if(sim["arv"]["tx"] < sim["dpr"]["tx"]) else "dpr",sim)

    #
    while( sim["arv"]["nx"] > sim["dpr"]["nx"]): dox("dpr", sim)  #nd, dep, nb, n, clock, td, td, tau2, maxT)

    prob   = {s:sim["nb"][s]/sim["clock"] for s in sim["nb"]}
    avgnb  = sum(s*prob[s] for s in prob)
    wait   = map(lambda x, y: x - y, sim["dpr"]["ex"], sim["arv"]["ex"])
    l      = list(wait)
    avgtps = sum(l)/len(l)
    return(round(avgnb,2),round(avgtps,2))

nb,tps = simul(2,3,100)
print("Average number of clients in the station:"+str(nb)+"  Average response  time: "+str(tps))
#______________________________   Output  ______________________________________
# 0.41408324302814 	 1 	 1
# 0.5497042464653211 	 2 	 2
# 0.6116388198560669 	 1 	 1
# 0.6296262864760898 	 0 	 2
# 0.6654205503868197 	 1 	 3
# 0.6916846815655274 	 2 	 4
# 0.848319495916065 	 3 	 5
#   ...
# Nombre moyen de clients dans la station:1.06  Temps de reponse moyen:0.6

0.3576569980752416 	 1 	 1
0.401398479284439 	 0 	 1
0.43179282346406206 	 1 	 2
0.5254020407689328 	 0 	 2
0.7926989395903434 	 1 	 3
0.9925517261909527 	 0 	 3
1.7466541529798518 	 1 	 4
1.8737828204123186 	 0 	 4
2.015431905850318 	 1 	 5
2.6156984810217443 	 0 	 5
2.843894075157239 	 1 	 6
3.239339712879068 	 2 	 7
3.3580923031965098 	 1 	 6
3.4646281285102476 	 0 	 7
3.6874472749229685 	 1 	 8
3.7190988705220183 	 0 	 8
3.9667523019634445 	 1 	 9
4.2306679729602275 	 0 	 9
5.244682245453867 	 1 	 10
5.508231192166715 	 0 	 10
5.570222546746394 	 1 	 11
5.6099920533809176 	 2 	 12
5.665218843940557 	 3 	 13
5.676845266805317 	 4 	 14
5.768156469647186 	 3 	 11
6.014179047884951 	 4 	 15
6.053353846317696 	 5 	 16
6.233966059174669 	 4 	 12
6.405820509414915 	 5 	 17
6.699134964097114 	 6 	 18
7.084192720921288 	 5 	 13
7.307680166084987 	 4 	 14
7.3197039018415655 	 5 	 19
7.71336024903381 	 4 	 15
7.840596181095036 	 3 	 16
8.16892534254493 	 2 	 17
8.200311948887162 	 1 	 18
8.23

In [0]:
#code701.py

from symbulate import BoxModel

# Die sampling
# this line could be replaced by :list(range(1, 6+1))
die = [1, 2, 3, 4, 5, 6]
roll = BoxModel(die)
print(roll.draw())

# draws 10 samples from ['A', 'B', 'C', 'D'] with probabilities [0.22, 0.27, 0.41, 0.10]
print(''.join(list(BoxModel(['A', 'B', 'C', 'D'], probs=[0.22, 0.27, 0.41, 0.10], size=10).draw())))

#______________________________   Output  ______________________________________
# 2
# CACCCCBAAB

In [0]:
#code702.py

from symbulate import BoxModel, ProbabilitySpace, Poisson, Exponential

#1 probabaility space with user defined function 
def gender_hobbies_sim():
    gender = BoxModel(["male", "female"], probs=[.2, .8]).draw()
    if gender == "male":
        hobbies = BoxModel(["science", "art", "sport"], probs=[.3, .3, .4]).draw()
    else:
        hobbies = BoxModel(["science", "art", "sport"], probs=[.2, .5, .3]).draw()
    return gender, hobbies
P = ProbabilitySpace(gender_hobbies_sim)
print(P.draw())

#2 independants probability spaces
die6, die4 = list(range(1,7,1)), list((1,5,1))
lancers = BoxModel(die6)*BoxModel(die4)
print(lancers.draw())

#3 independants probability spaces
three = BoxModel(['P','F'])*Poisson(lam=3)*Exponential(rate=4)
print(three.draw())

#______________________________   Output  ______________________________________
#('female', 'art')
#(5, 1)
#(F, 1, 0.029286540827854368)

In [0]:
#Code703.py

from symbulate import BoxModel, RV, Normal, exp

# 5 Bernoulli trials and successes number
P = BoxModel([0,1],size=5); X = RV(P,sum)
tab = (X>3).sim(10000).tabulate()
print("Tabulate without normalization: ",tab)
tab = (X>3).sim(10000).tabulate(normalize=True)
print("Tabulate with normalization: ",tab)
X.sim(10000).plot()

# function of rv
X = RV(Normal(mean=0, var=1)); Y = exp(X) # could be replaces with: X.apply(exp)
Y.sim(10000).plot()

#______________________________   Output  ______________________________________
# Tabulate without normalization:  {True: 1826, False: 8174}
# Tabulate awith normalization:  {True: 0.1842, False: 0.8158}

In [0]:
#Code704.py

from symbulate import RV, Normal, Exponential

X = RV(Exponential(rate=1/4))
simX = X.sim(10000)
print("Expected value:", simX.mean())
print("Variance:", simX.var())
print("Standard deviation:", simX.sd())

#Normalization
X = RV(Normal(mean=3,sd=2))
simX = X.sim(10000)
print("Before normalization. mean and sd:", round(simX.mean(),2), round(simX.sd(),2))
simX.plot()

z = simX.standardize()
print("After normalization. mean and sd:", round(z.mean(),2), round(z.sd(),2))
z.plot()

#______________________________   Output  ______________________________________
# Expected value: 3.9628993353781383
# Variance: 15.587130335771537
# Standard deviation: 3.9480539935228265
# Before normalization. mean and sd: 3.03 2.0
# After normalization. mean and sd: -0.0 1.0

In [0]:
# Code705.py

from scipy.stats import expon
import matplotlib.pyplot as plt

def generatePoissonSP(lamda=1, T=1):
    t, N, tim = expon.rvs(scale=1/lamda), 0, []
    while (t<T): 
        tim.append(t)
        N, t = N+1, t+expon.rvs(scale=1/lamda)
    return N,tim

def plotPP(lamda):
    n,tim = generatePoissonSP(lamda,20)
    return plt.step(tim,range(n))

lamdas = [0.25,0.5,1,2]
plt.legend([plotPP(l)[0] for l in lamdas],["lambda="+str(l) for l in lamdas])
plt.xlabel('Time'); plt.ylabel('arrivals number')
plt.show()

In [0]:
# Code 706.py

from itertools import accumulate
from random import random
import sys;sys.path.append('../chap4')
from CMTD import CMTD

def simulateCM(P,S,steps,cs):
    CM = CMTD(P,S)
    ls = []
    indices = range(len(CM.S))
    for i in range(steps):
        ls.append(CM.S[cs])
        prob = list(accumulate(P[cs]))
        cs=transformation_inverse(prob,indices)
    return (CM, "".join(ls))

def transformation_inverse(prob,indices):
    u = random()
    for i in range(len(prob)):
        if(u<prob[i]): return indices[i]
    return indices[-1]

def compareToSteadyState(P,S,n,s0):
    res = simulateCM(P,S,n,s0)
    ana_p = res[0].steady_prob();           print("Analytical prob : ", ana_p)
    emp_p = [res[1].count(s)/n for s in S]; print("Empirical prob  : ", emp_p)

# Test
n = 10000
print('Ergodic')
P = [[1/3, 1/3, 1/3],
     [3/5, 0.0, 2/5],
     [3/4, 1/8, 1/8]]
S = ['R','N','S']

compareToSteadyState(P,S,n,1)
n = 50
print("CMTD1: ",simulateCM(P,S,n,1)[1])

print('Not ergodic')
P = [[1.0, 0.0, 0.0],
     [3/5, 0.0, 2/5],
     [3/4, 1/8, 1/8]]
print("CMTD2: ",simulateCM(P,S,n,1)[1])

#______________________________   Output  ______________________________________
# Ergodic
# Analytical prob :  [0.50769231 0.20512821 0.28717949]
# Empirical prob  :  [0.5065, 0.2072, 0.2863]
# CMTD1:  NRNRSRSRNSRRRNRSRNRNRSRRSNRRNRSRRSRNSNRSSRRNRSRNRR
# Not ergodic
# CMTD2:  NRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRRR

In [0]:
# Code 707.py

from itertools import accumulate
from random import random
import math
import sys;sys.path.append('../chap4')
from CMTC import CMTC
 
expo = lambda lamda : (-1/lamda)*math.log(random())

def transformation_inverse(prob,indices):
    U = random()
    for i in range(len(prob)):
        if(U < prob[i]): return indices[i]
    return indices[-1]

def simulateCM(P,lamdas,S,T_max,cs):
    CM = CMTC(P,lamdas,S)  
    indices = range(len(CM.S))
    ls, tc = [], 0
    while tc < T_max:
        tps = expo(lamdas[cs])
        ls.append((CM.S[cs], round(tps,2)))
        tc = tc + tps
        prob = list(accumulate(P[cs]))
        cs = transformation_inverse(prob,indices)
    return (CM , ls)

def compareToSteadyState(res,T):
    ana_p = [ round(p,2) for p in res[0].steady_prob()]
    print("Analytical prob : ", ana_p)
    emp_p = [ round(sum(map(lambda r: r[1], filter(lambda r: r[0]==s, res[1])))/T,2) for s in S]
    print("Empirical prob  : ", emp_p)

S = ['R','N','S']
P = [[0.0, 2/3, 1/3],
     [3/5, 0.0, 2/5],
     [3/4, 1/4, 0.0]]
lamdas = [4, 5, 6]
T_max = 500

res = simulateCM(P,lamdas, S,T_max,1)
compareToSteadyState(res, T_max)
print("CMTC : ",simulateCM(P,lamdas,S,5,1)[1])

#______________________________   Output  ______________________________________
# Analytical prob :  [0.47, 0.32, 0.21]
# Empirical prob  :  [0.46, 0.33, 0.22]
# CMTC : [('N', 0.05), ('R', 0.0), ('S', 0.03),..., ('S', 0.09), ('R', 0.19)]

In [0]:
# Code708.py

import random
import math
import matplotlib.pyplot as plt
import sys;sys.path.append('../chap5')
from Code502 import getMM1

expo = lambda lmbd : (-1/lmbd)*math.log(random.random())

#
def simul(lamda,mu,maxT):
    clock, n, nb = 0, 0, {}             # n is customers number in the system
    t_arv, t_dep = expo(lamda), maxT    # next arrival/departure/max time
    arv, dep = [], []                   # Arrival/Departure times
    n_arv, n_dep = 0, 0                 # Arrivals/Departures number

    #
    def departure():
        nonlocal n_dep, dep, n, t_dep, mu, maxT, clock

        n_dep += 1;  dep.append(t_dep)
        if(n >= 0): nb[n] = (nb[n] if n in nb else 0) + t_dep - clock
        n, clock = n-1, t_dep
        t_dep = clock + expo(mu) if n>0 else maxT

    stats = []
    #
    while(clock < maxT):
        stats.append((clock,n))           # add event id and time
        if(t_arv < t_dep):                # arrival event

            n_arv += 1; arv.append(t_arv)
            nb[n] = (nb[n] if n in nb else 0) + t_arv - clock
            n, clock = n+1, t_arv
            t_arv = clock + expo(lamda)

            if(n == 1): t_dep = clock + expo(mu)
        else:                             # departure event
            departure()

    #
    while(n_arv > n_dep):
        stats.append((clock,n))
        departure()

    prob = {s:nb[s]/clock for s in nb}
    avg_nb_clients = sum(s*prob[s] for s in prob)
    wait = list(map(lambda x, y: x - y, dep, arv))
    avg_time = sum(wait)/len(wait)
    return(avg_nb_clients,avg_time, stats)

#
print('Analytical performances : ');   getMM1(2,3)
print('Empirical performances : ');     nb, tps, stats =simul(2,3,10)
print("Average number of clients in the station: "+str(round(nb,2)))
print("Average response time: " + str(round(tps,2)))

# Plot
plt.step([t[0] for t in stats], [t[1] for t in stats])
plt.xlabel('Time'); plt.ylabel('clients number')
plt.show()
#______________________________   Output  ______________________________________
# Performances ananlytiques : 
# ==================MM1 =======================
# p0* :0.33333
# p1* :0.22222
# p3* :0.09877
# L   :2.00
# Lq  :1.33
# W   :1.00
# Wq  :0.67
# Performances empiriques : 
# Nombre moyen de clients dans la station:1.97
# Temps de reponse moyen:0.98

In [0]:
# Code709.py

import random
import simpy
from simpy.resources import resource

# Shared
e =  simpy.Environment()
P = e.process
R = e.run

# Resources
RSC = simpy.Resource
RSC_withCapacity = lambda cap: simpy.Resource(e, capacity=cap)
RSC_Prempt =  lambda cap: resource.PreemptiveResource( e , capacity = cap )
RSC_Prio =  lambda cap: resource.PriorityResource( e , capacity = cap )
RQT_prio = resource.PriorityRequest

TO = e.timeout
SETO =  simpy.events.Timeout

# load datastructures
ld  = { 'Ls':[],     # Load historique in the system
        'L':[0],     # current load
        'Q':[0],     # current queue
        'tl':[0],    # time when updating the load
        'tq':[0],    # time when updating the queue
        'Nfs':[1]    # Number of servers
        }

# updateLoad L/Q
def updateLoad(E, te, h,k, stationId=0):
    global ld
    ld[E][stationId] = ld[E][stationId] + h;     # number of client in Queue Q or System L
    ld['Ls'].append((round(e.now - ld[te][stationId],4), ld[E], k)); ld[te][stationId] =e.now

#
rexpo = lambda mu : random.expovariate(mu)
rinverse = lambda probas : random.choices(range(len(probas)), probas)[0]
runif = lambda a,b : random.randint(a, b)
randColor = lambda:(random.random(),random.random(),random.random())

# iterator for the loop
def geti():
    i=0;
    while True: i=i+1; yield i
g = geti()

# on Event Arrival +  Sertvice  +  Departure
def onEvent(event, id, stats, stationId=0):
    typEvent = event[0]
    # update system load 
    if typEvent  != 'S': updateLoad('L' ,'tl' ,(1 if typEvent=="A" else -1) ,0, stationId)

    # update system queue
    if ((typEvent  == 'A' and ld['Nfs'][stationId] ==  0) or
        (typEvent  == 'S' and ld['Q'][stationId] > 0)) :
        updateLoad('Q' ,'tq' ,(1 if typEvent=="A" else -1) ,1, stationId)

    # update number of available servers
    if typEvent  != 'A': 
        ld['Nfs'][stationId]= ld['Nfs'][stationId] + (1 if typEvent=="D" else -1)

    #statistic array ['E', time, 'ID', servers , L ,Q ,Station]
    stats.append(
        [ typEvent,                       # Event type  Arrival, Service and Departure
          round(e.now,4),                 # time
          id,                             # client id 'ClientID'
          ld['Nfs'][stationId],           # number of available servers
          ld['L'][stationId],             # station load
          ld['Q'][stationId],             # station queue
          stationId                       # station id
          ]
        )

# ===========================================================================
# wait_queue   +  wait_system  +
def process_output(psim):
    wait_queue, wait_system, N = 0, 0, len(psim)
    for evt in psim:
        wait_queue   += evt[2] - evt[1]
        wait_system  += evt[3] - evt[1]
    wait_queue   = round(wait_queue /N,4)
    wait_system  = round(wait_system/N,4)
    return (wait_queue, wait_system)

#
def plot_state(stats0):
    import matplotlib.pyplot as plt
    nbstations =  len(stats0)
    for i in range(nbstations):
        plt.step([v[0] for v in stats0[i].values()], [v[3] for v in stats0[i].values()])
    plt.xlabel('Time');  plt.ylabel('clients number')
    plt.show()

# Statistics
def collect_stats(psim, pld, show, nbstations=1):
    stats0 =[{} for _ in range(nbstations)]
    stats1 =[{} for _ in range(nbstations)]
    eprev, L, Lq = 0, 0, 0

    for currentStation in range(len(stats0)):
        stats = stats0[currentStation];  statsW = stats1[currentStation]
        psim0 = [e for e in psim['staE'] if e[6] == currentStation]  # filter stats for the current station

        # collect stats [time, [('E', 'ID'),...], Server, Q, L, Station]
        for evt in psim0:
            # stats for the system based on time as key
            if not evt[1] in stats.keys():  # evt[1] time
                stats[evt[1]] = [evt[1], [],0, 0, 0,0]
                stats[evt[1]][2] = evt[3]   # Available servers
                stats[evt[1]][3] = evt[4]   # L
                stats[evt[1]][4] = evt[5]   # Q
            else:
                stats[evt[1]][2] = min(stats[evt[1]][2], evt[3])
                stats[evt[1]][3] = min(stats[evt[1]][3], evt[4])
                stats[evt[1]][4] = min(stats[evt[1]][4], evt[5])
            stats[evt[1]][1].append((evt[0], evt[2]))

            # stats for the system queue : W based on client_ID [ID, A, S, D]
            if evt[2] != 'None':   # evt[2] Client ID
                if not evt[2] in statsW.keys(): statsW[evt[2]] = [evt[2], 0, 0, 0]
                if evt[0]=='A': statsW[evt[2]][1] = evt[1]
                if evt[0]=='S': statsW[evt[2]][2] = evt[1]
                if evt[0]=='D': statsW[evt[2]][3] = evt[1]

            L += (evt[1] - eprev)*evt[4]
            Lq += (evt[1] - eprev)*evt[5]
            eprev = evt[1]

        wqs = process_output(statsW.values())
        lds = (round(L/psim['T_sim'],4), round(Lq/psim['T_sim'],4))
        return wqs, lds, stats0

# show_statistics
def show_statistics(psim, pld, show, nbstations=1):
    wqs, lds, sta = collect_stats(psim, pld, show, nbstations)#
    if show :
        for e in sta:
            for v in e.values() : print(v)

    print("Empirical performances :")
    print("Average time in the queue               : ", wqs[0])
    print("Average time in the system              : ", wqs[1])
    print("Average number of clients in the queue  : ", lds[1])
    print("Average number of clients in the system : ", lds[0])
    plot_state(sta)

# ===========================================================================
# method that plots the gant chart of the application
def gant(cpus, tasks,l, H=50, M=5 ):
    import matplotlib.pyplot as plt
    fig, axi = plt.subplots()
    axi.set_ylim(0, H);      axi.set_xlim(0, l)
    axi.set_xlabel('Time');  axi.set_ylabel('Processor')
    axi.grid(True)

    # Setting ticks on y-axis
    dx =  int((H - 2*M)/ len(cpus))
    axi.set_yticks([dx + i*dx  for i in range(len(cpus))])
    axi.set_yticklabels(cpus)

    # Declaring a bar in schedule
    for k,task in tasks.items():
        posH=(task['cpu']+1)*dx-M/2
        axi.broken_barh(task['length'], (posH, M) , facecolors = task['color'])
        for t in task['length']:
            axi.annotate(k, (1,1), xytext=(t[0]+t[1]/2-1, posH+M+1))
    plt.show()

In [0]:
# Code710.py

from Code709 import e,P,R,TO, RSC, rexpo, g, show_statistics, ld, onEvent
import sys;sys.path.append('../chap5')

# =========================================================================
# station entity model
class Station(object):
    def __init__(self, psim):
        self.machines = RSC(e, psim['S']);
        self.mu = psim['mu']; self.sd = psim['SD']

    def wash(self, voit):
        yield TO(self.sd(self.mu))

# car entity model
def Car(name, station, psim):
    # event Arrival, Service, Departure
    onCarEvent = lambda E: onEvent(E, name, psim['staE'])

    # core
    onCarEvent('Arrival')                                    # onArrival event
    with station.machines.request() as machine:
        yield machine;               onCarEvent('Service')   # onService event
        yield P(station.wash(name)); onCarEvent('Departure') # onDeparture event

# simulation setup
def setup(psim):
    station = Station(psim)
    ld['Nfs']= [psim['S']]
    psim['staE'].append(['I', 0.0, 'None', psim['S'] ,0 ,0 ,0])      # initilize statistics
    while True:
        yield TO(psim['AD'](psim['lambda']))
        if psim['K']==-1 or len(station.machines.queue) < psim['K']: # Queue Capcity limit
            if(e.now > psim['T_sim']): break                         # Simulation time limit
            P(Car('C%d' % next(g), station, psim))                   # new arrival process

# =============================================================================

def run_sim(sim, show=False):
    P(setup(sim)); R()                                    # Simulation
    show_statistics(sim, ld, show)                        # Statistic
#
def sim_mmsk(show=False):
    from Code505 import getMMSK

    # simulation examples
    sim = {
    'T'     : 'MMSK',      # type of the model
    'mu'    : 4,           # service rate
    'lambda': 10,          # arrival rate
    'S'     : 3,           # number of servers
    'K'     : 30,          # Queue capacity
    'SD'    : rexpo ,      # service distribution
    'AD'    : rexpo,       # arrival distribution
    'T_sim' : 1000,        # Simulation time
    'staE'  : [] }         # Statistics array

    run_sim(sim, show)                                      # simulation
    getMMSK(sim['mu'],sim['lambda'],sim['S'], sim['K'])     # Theory

sim_mmsk()


# def sim_md1(show=False):
#     simulation de la station M/D/1
#     sim = {
#     'T':'MDS',              # type of the model
#     'mu':8,                 # service rate
#     'lambda':6,             # arrival rate
#     'S':1,                  # number of servers
#     'K':-1,                 # Queue capacity
#     'SD': lambda x:1/x ,    # service distribution
#     'AD':rexpo,             # arrival distribution
#     'T_sim':10000,          # Simulation time
#     'staE':[] }             # Statistics array

#     run_sim(sim, show)                                        # simulation
#     print("Performances analytiques :")
#     D , rho = 1/sim['mu'],  sim['lambda']/sim['mu']
#     Wq      = rho/((2/D)*(1-rho));
#     W       = D + Wq ;
#     Lq      = 1/2*rho**2/(1-rho);
#     L       = rho + Lq;
#     print("Temps d'attente moyen dans la file      : ", Wq)
#     print("Temps d'attente moyen dans la station   : ", W)
#     print("Nombre moyen de clients dans la file    : ", Lq)
#     print("Nombre moyen de clients dans la station : ", L)

# sim_md1()

In [0]:
#Code711.py

import heapq  # heapqueue allows to allocate servers in round robin policy
from Code709 import e,P,R,TO, rexpo, g, rinverse, RSC_withCapacity, SETO

# ===============================================================================================
# show
def show(cz, args):
    if cz == 2 : print('%7.2f \t %s \t %s \t %s \t %s' % tuple([e.now] + args))
    else       : print('%7.2f \t %s \t %s \t\t %s' % tuple([e.now] + args))

# main method
def carwash(T, topology):
    class Station(object):
        def __init__(self, id, infos):
            self.id=id;
            self.S= infos['S']
            self.M = infos['Mu']
            self.nextStations= infos['nextS']
            self.probas=infos['probas']
            self.Indices = list(range(self.S));
            self.Servers = RSC_withCapacity(self.S);

        def pf_svc(self, c): X =rexpo(self.M);   yield TO(X)

        # onService
        def onService(self, id):
            if len(self.Indices) > 0:
                svr = heapq.heappop(self.Indices);    # pop in queue
                show(2, [id, 'S', svr, self.id])
            return svr

        # onDeparture
        def onDeparture(self,id, svr):
            heapq.heappush(self.Indices, svr);                       # push from queue
            show(1, [id, 'D',self.id])

        # pf_getin : when access a station
        def pf_getin(self, id):
            show(1, [id, 'A', self.id])                              # Arrival
            with self.Servers.request() as s:
                yield s;                   svr = self.onService(id)  # Service
                yield P(self.pf_svc(id));  self.onDeparture(id, svr) # Departure
                if self.nextStations != None:
                    i =  rinverse(self.probas)
                    if(i<len(self.nextStations)):
                        P(self.nextStations[i].pf_getin(id)) # go to next station

    # create_system : create the topology
    def create_system(topo):
        stations = [ Station(i,            #  the station's ID
                    { 'S': topo[i][0][0],  # number of servers
                      'Mu': topo[i][0][1], # service rate
                      'nextS':None ,       # list of next stations
                      'probas': topo[i][2] # probabilities to go to next stations
                     })
                     for i in range(len(topo)) if i!=0]
        # create next stations
        for station in stations:
            if(topo[station.id][1]!=None):
                station.nextStations = [stations[v-1] for v in topo[station.id][1]]

        # returns initial stations with their arrival rates
        return [stations[i-1] for i in topo[0][1]] , topo[0][2]

    # setup the simulation
    def setup():
        def firstS(lamdas):
            t=min(times)
            first=times.index(t)
            for k in range(len(times)): times[k]-=t
            times[first]=rexpo(lamdas[first])
            return first,t

        initStations, lamdas = create_system(topo)

        times=[0 for i in range(len(initStations))]
        for i in range(len(times)): times[i]= rexpo(lamdas[i])
        while e.now<T:
            indf, t=firstS(lamdas)
            car = yield SETO(e, delay=t, value='C%d' % next(g))
            P(initStations[indf].pf_getin(car))

    P(setup());  R()

# Test
topo = [
  #(Svr_Nb,mu)  [successors], [Probavility]
  ((None,None), [1,3], [6,2]),
  ((2   ,4   ), [2,3], [0.6,0.2,0.2]),
  ((1   ,3   ), [3]  , [0.5,0.5]),
  ((1   ,6   ), None , None)]

carwash(5,topo)

In [0]:
# Code712.py

from simpy import Interrupt
from Code709 import e,P,R,TO, rexpo, RSC_Prempt, RQT_prio, RSC_Prio

#
def station(nbClients,serveurs,lamdas,mu,stat, preemption=True):
    times = [ rexpo(l) for l in lamdas ]          # arrival times initialization
    for id in range(nbClients):
        t = min(times)                            # selects the earlier arrival
        priority = times.index(t)                 # gets its priority
        times = [ tc - t for tc in times ]        # updates the arrival times
        times[priority] = rexpo(lamdas[priority]) # generate the next arrival of this priority
        yield TO(t)
        stat[priority][id]=[0,0,0]
        P(client(id,priority,serveurs,mu,stat, e.now, preemption))

#
def client(id,priority,serveurs,mu,stat, beginTime, preemption=True):
    try:
        stat[priority][id][0] = beginTime
        with RQT_prio(serveurs,priority, preempt=preemption) as req:
            req.time=beginTime
            yield req
            stat[priority][id][1] = e.now
            yield TO(rexpo(mu))
            stat[priority][id][2] = e.now
    except Interrupt:   # in case of preemptive priority
        P(client(id,priority,serveurs,mu,stat, beginTime, preemption))

#
def statistiques(stat):
    rgs = range(len(stat))
    wait_queue, wait_system = [0 for _ in rgs], [0 for _ in rgs]
    for priority in rgs:
        for k in stat[priority].keys():
            wait_queue[priority] += stat[priority][k][1] - stat[priority][k][0]
            wait_system[priority]  += stat[priority][k][2]- stat[priority][k][0]
    print("Wq : Average waiting time in the queue")
    print("W  : Average waiting time in the station")

    print("priority \t  Wq \t\t  W ")
    for p in rgs:
        twq = round(wait_queue[p]/len(stat[p]),2)
        tw = round(wait_system[p]/len(stat[p]),2)
        print(p," \t\t\t ", twq, " \t ", tw)

#
nbClients, nbServeurs, mu, lamdas, preemption = 100, 1 ,10, [3,4,2], False
stat = [{} for p in range(len(lamdas))]

serveurs = (RSC_Prempt if preemption else RSC_Prio)(nbServeurs)
P(station(nbClients, serveurs, lamdas, mu, stat, preemption)); R()

#
print("Empirical performances:")
statistiques(stat)

In [0]:
#Code713.py

import simpy
from Code709 import (e,P,R,TO, rexpo, runif, g, RSC_Prempt, SETO,
                     RQT_prio, gant, randColor)

remain = lambda X,start,speed: X - (e.now - start)*speed # remaining time in the server
# show
def show(evt, s, task, X):
    print(round(e.now,3), " \t " + evt + " \t ",task[0], " \t ", X ,
          " \t ", s.id, " \t ", task[2], " \t ", end='')

class Station(object):
        def __init__(self, id, infos):
            self.id = id;
            self.Speed = infos['Speed']
            self.Servers = RSC_Prempt(1);

        def update(self, task, X, dicTask, evnt, start):
            X = int(remain(X,start,self.Speed))
            dicTask[task[0]]['length'].append((round(start,3),round(e.now-start,3)))
            show(evnt, self, task, X)
            return X

        def pf_svc(self, X): delay = X/self.Speed ;   yield TO(delay)

        def pf_run(self, task, stations, X, dicTask, preemption=True):
            if X>0:
                try:
                    with RQT_prio(self.Servers,task[2], preempt=preemption) as s:
                        print([len(s.Servers.queue) for s in stations])
                        yield s;
                        start = e.now
                        show('S', self, task, X) ; print()
                        yield P(self.pf_svc(X))
                        X = self.update(task, X, dicTask, 'D', start)
                    print([len(s.Servers.queue) for s in stations])
                except simpy.Interrupt:
                    if remain(X,start,self.Speed)>0:
                        X = self.update(task, X, dicTask, 'I', start)
                        P(self.pf_run(task,stations, X, dicTask, preemption))

# create the system: one station for each processor
def create_system(proc):
        stations = [ Station(i,{'Speed': proc[i]}) for i in range(len(proc))]
        return stations

#
def schedule(T,lamda, procs, pmin, pmax, policy, dicTask):
    print('Time \t Event \t Task \t Remain \t Server \t Priority \t Queues')
    stations = create_system(procs)
    while e.now<T:
        t = rexpo(lamda) # next arrival
        # creates a new task (Id, length, priority)
        task = yield SETO(e, delay=t, value=('T%d' % next(g), runif(pmin,pmax)*10, runif(1,3)))
        proc = policy(stations) # calls the specified policy to selects a processor
        # intialize execution structure of the task
        dicTask[task[0]] = {'length':[], 'cpu':proc, 'color':randColor()}
        show('A', stations[proc] , task, task[1])
        # creates the station process associated to the current task
        P(stations[proc].pf_run(task, stations, task[1],  dicTask ))

# lessLoaded policy
def lessLoaded(stations):
    lp = list( map(lambda s: s.Servers.count, stations))
    if(min(lp) == 0): return lp.index(min(lp))
    lq = list( map(lambda s: len(s.Servers.queue), stations))
    return lq.index(min(lq))

# processors speeds, tasks dictionary
processors, dicTask = [3,2,4], {}
# pmin, pmax: minimum and maximum length of tasks, T: simulation time
lamda, pmin, pmax, T = 5, 1, 9, 2
P(schedule(T,lamda,processors, pmin, pmax, lessLoaded, dicTask));  R()

for k,t in dicTask.items():
    print(k," cpu:",t['cpu'],'  length:',t['length'])
lastD = max([d[0]+d[1] for t in dicTask.values() for d in t['length']])
gant(range(len(processors)),dicTask, int(lastD)+10)

#______________________________   Output  ______________________________________
# T1  cpu: 0   length: [(0.232, 1.872), (25.438, 21.333)]
# T2  cpu: 1   length: [(0.33, 0.647), (50.976, 24.0)]
# T3  cpu: 2   length: [(0.655, 22.5)]
# T4  cpu: 0   length: [(5.438, 20.0)]
# T5  cpu: 1   length: [(0.976, 20.0)]
# T6  cpu: 2   length: [(23.155, 12.5)]
# T7  cpu: 0   length: [(46.771, 6.667)]
# T8  cpu: 1   length: [(20.976, 30.0)]
# T9  cpu: 2   length: [(35.655, 15.0)]
# T10  cpu: 0   length: [(2.104, 3.333)]

In [8]:
# method that plot the gant chart of the application
def gant(cpus, tasks,l, H=50, M=5 ):
    import matplotlib.pyplot as plt
    fig, axi = plt.subplots()
    axi.set_ylim(0, H);      axi.set_xlim(0, l)
    axi.set_xlabel('Time');  axi.set_ylabel('Processor')
    axi.grid(True)

    # Setting ticks on y-axis
    dx =  int((H - 2*M)/ len(cpus))
    axi.set_yticks([dx + i*dx  for i in range(len(cpus))])
    axi.set_yticklabels(cpus)

    # Declaring a bar in schedule
    for k,task in tasks.items():
        posH=(task['cpu']+1)*dx-M/2
        axi.broken_barh(task['length'], (posH, M) , facecolors = task['color'])
        for t in task['length']:
            axi.annotate(k, (1,1), xytext=(t[0]+t[1]/2-1, posH+M+1))
    plt.show()

In [0]:
#Code713.py

import simpy
from Code709 import (e,P,R,TO, rexpo, runif, g, RSC_Prempt, SETO,
                     RQT_prio, gant, randColor)

remain = lambda X,start,speed: X - (e.now - start)*speed # remaining time in the server
# show
def show(evt, s, task, X):
    print(round(e.now,3), " \t " + evt + " \t ",task[0], " \t ", X ,
          " \t ", s.id, " \t ", task[2], " \t ", end='')

class Station(object):
        def __init__(self, id, infos):
            self.id = id; 
            self.Speed = infos['Speed']
            self.Servers = RSC_Prempt(1);

        def update(self, task, X, dicTask, evnt, start):
            X = int(remain(X,start,self.Speed))
            dicTask[task[0]]['length'].append((round(start,3),round(e.now-start,3)))
            show(evnt, self, task, X)
            return X

        def pf_svc(self, X): delay = X/self.Speed ;   yield TO(delay)

        def pf_run(self, task, stations, X, dicTask, preemption=True):
            if X>0:
                try:
                    with RQT_prio(self.Servers,task[2], preempt=preemption) as s:
                        print([len(s.Servers.queue) for s in stations])
                        yield s;
                        start = e.now
                        show('S', self, task, X) ; print()
                        yield P(self.pf_svc(X))
                        X = self.update(task, X, dicTask, 'D', start)
                    print([len(s.Servers.queue) for s in stations])
                except simpy.Interrupt:
                    if remain(X,start,self.Speed)>0:
                        X = self.update(task, X, dicTask, 'I', start)
                        P(self.pf_run(task,stations, X, dicTask, preemption))

# create the system: one station for each processor
def create_system(proc):
        stations = [ Station(i,{'Speed': proc[i]}) for i in range(len(proc))]
        return stations

#
def schedule(T,lamda, procs, pmin, pmax, policy, dicTask):
    print('Time \t Event \t Task \t Remain \t Server \t Priority \t Queues')
    stations = create_system(procs)
    while e.now<T:
        t = rexpo(lamda) # next arrival
        # creates a new task (Id, length, priority)
        task = yield SETO(e, delay=t, value=('T%d' % next(g), runif(pmin,pmax)*10, runif(1,3)))
        proc = policy(stations) # calls the specified policy to selects a processor
        # intialize execution structure of the task
        dicTask[task[0]] = {'length':[], 'cpu':proc, 'color':randColor()}
        show('A', stations[proc] , task, task[1])
        # creates the station process associated to the current task
        P(stations[proc].pf_run(task, stations, task[1],  dicTask ))

# lessLoaded policy
def lessLoaded(stations):
    lp = list( map(lambda s: s.Servers.count, stations))
    if(min(lp) == 0): return lp.index(min(lp))
    lq = list( map(lambda s: len(s.Servers.queue), stations))
    return lq.index(min(lq))

# processors speeds, tasks dictionary
processors, dicTask = [3,2,4], {}
# pmin, pmax: minimum and maximum length of tasks, T: simulation time
lamda, pmin, pmax, T = 5, 1, 9, 2
P(schedule(T,lamda,processors, pmin, pmax, lessLoaded, dicTask));  R()

for k,t in dicTask.items():
    print(k," cpu:",t['cpu'],'  length:',t['length'])
lastD = max([d[0]+d[1] for t in dicTask.values() for d in t['length']])
gant(range(len(processors)),dicTask, int(lastD)+10)

#______________________________   Output  ______________________________________
# T1  cpu: 0   length: [(0.232, 1.872), (25.438, 21.333)]
# T2  cpu: 1   length: [(0.33, 0.647), (50.976, 24.0)]
# T3  cpu: 2   length: [(0.655, 22.5)]
# T4  cpu: 0   length: [(5.438, 20.0)]
# T5  cpu: 1   length: [(0.976, 20.0)]
# T6  cpu: 2   length: [(23.155, 12.5)]
# T7  cpu: 0   length: [(46.771, 6.667)]
# T8  cpu: 1   length: [(20.976, 30.0)]
# T9  cpu: 2   length: [(35.655, 15.0)]
# T10  cpu: 0   length: [(2.104, 3.333)]

<center><img src="./fig713.png" width=500px /></center>